A metaclass is defined by inheriting from type. In the default case, a metaclass receives the contents of associated class statements in its new method. Here, I can inspect and modify the class information before the type is actually consturcted

In [13]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        print(f'* Running {meta}.__new__ for {name}')
        print('Bases:', bases)
        print(class_dict)
        return type.__new__(meta, name, bases, class_dict)
    
    
class MyClass(metaclass=Meta):
    stuff = 123
    
    def foo(self):
        pass
    
class MySubclass(MyClass):
    other = 567
    
    def bar(self):
        pass

* Running <class '__main__.Meta'>.__new__ for MyClass
Bases: ()
{'__module__': '__main__', '__qualname__': 'MyClass', 'stuff': 123, 'foo': <function MyClass.foo at 0x000002B0C915BC70>}
* Running <class '__main__.Meta'>.__new__ for MySubclass
Bases: (<class '__main__.MyClass'>,)
{'__module__': '__main__', '__qualname__': 'MySubclass', 'other': 567, 'bar': <function MySubclass.bar at 0x000002B0C7472C20>}


It can add functionality to the Meta.__new__ method in order to validate all of the parameters of an associated class before it's defined

In [6]:
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        #only validate subclasses of the Polygon class
        if bases:
            if class_dict['sides'] < 3:
                raise ValueError('Polygons need 3+ sides')
        return type.__new__(meta, name, bases, class_dict)
    
    
class Polygon(metaclass=ValidatePolygon):
    sides = None #must be specified by subclass
    
    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180
    
    
class Triangle(Polygon):
    sides = 3
    

class Rectangle(Polygon):
    sides = 4
    

class Nonagon(Polygon):
    sides = 9
    
    
assert Triangle.interior_angles() == 180
assert Rectangle.interior_angles() == 360
assert Nonagon.interior_angles() == 1260


In [7]:
print('Before class')

class Line(Polygon):
    print('Before sides')
    sides=  2
    print('After sides')
    
print('After class')

Before class
Before sides
After sides


ValueError: Polygons need 3+ sides

Python 3.6 intorduced simplified syntax -- the\_\_init\_subclass\_\_ special class method -- for achieving the same behavior while avoiding metaclasses entirely

In [8]:
class BetterPolygon:
    sides = None  # Must be specified by subclasses
    
    def __init_subclass__(cls):
        super().__init_subclass__()
        if cls.sides < 3:
            raise ValueError('Polygons need 3+ sides')
            
    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180
    
    
class Hexagon(BetterPolygon):
    sides = 6
    
assert Hexagon.interior_angles() == 720

In [9]:
print('Before class')

class point(Polygon):
    print('Before sides')
    sides=  1
    print('After sides')
    
print('After class')

Before class
Before sides
After sides


ValueError: Polygons need 3+ sides

Another problem with the standard Python metaclass machinery ia that you can only specify a single metaclass per class definition.

In [10]:
class ValidateFilled(type):
    def __new__(meta, name, bases, class_dict):
        #only validate subclasses of the filled class
        if bases:
            if class_dict['color'] not in ('red', 'green'):
                raise ValueError('Fill color must be supported')
        return type.__new__(meta, name, bases, class_dict)
    
    
class Filled(metaclass=ValidateFilled):
    color = None # must be specified by subclasses
    
class RedPentagon(Filled, Polygon):
    color = 'red'
    sides = 5

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

In [25]:
#it's possible to fix this by creating a complex hierarchy of metaclass type definitions to layer validation
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        print(f'in ValidatePolygon * Running {meta}.__new__ for {name}')
        print(class_dict)
        #Only validate non-root classes
        if not class_dict.get('is_root'):
            if class_dict['sides'] < 3:
                raise ValueError('Polygons need 3+ sides')
                
        return type.__new__(meta, name, bases, class_dict)
    
    
"""
class Polygon(metaclass=ValidatePolygon):
    is_root = True
    sides = None # Must be specified by subclasses"""
    
class ValidateFilledPolygon(ValidatePolygon):
    def __new__(meta, name, bases, class_dict):
        print(f'in ValidateFilledPolygon * Running {meta}.__new__ for {name}')
        #Only validate non-root classes
        if not class_dict.get('is_root'):
            if class_dict['color'] not in ('red', 'green'):
                raise ValueError('Fill color must be supported')

        return super().__new__(meta, name, bases, class_dict)

class FilledPolygon( metaclass=ValidateFilledPolygon):
    is_root = True
    color = None
    sides = None
    
class GreenPentagon(FilledPolygon):
    color = 'green'
    sides = 5

in ValidateFilledPolygon * Running <class '__main__.ValidateFilledPolygon'>.__new__ for FilledPolygon
in ValidatePolygon * Running <class '__main__.ValidateFilledPolygon'>.__new__ for FilledPolygon
{'__module__': '__main__', '__qualname__': 'FilledPolygon', 'is_root': True, 'color': None, 'sides': None}
in ValidateFilledPolygon * Running <class '__main__.ValidateFilledPolygon'>.__new__ for GreenPentagon
in ValidatePolygon * Running <class '__main__.ValidateFilledPolygon'>.__new__ for GreenPentagon
{'__module__': '__main__', '__qualname__': 'GreenPentagon', 'color': 'green', 'sides': 5}


In [29]:
class Filled:
    color = None
    
    def __init_subclass__(cls):
        super().__init_subclass__()
        if cls.color not in ('red', 'green', 'blue'):
            raise ValueError('Fills need a valid color')
            
            
#it can inherit form both classes to define a new class. Both
#classes call super().__init_subclass__(), causing their 
#corresponding validation logic to run when the subclass is create

class RedTriangle(Filled, BetterPolygon):
    color = 'red'
    sides = 3
    
    
ruddy = RedTriangle()
assert isinstance(ruddy, Filled)
assert isinstance(ruddy, BetterPolygon)

In [30]:
class Top:
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Top for {cls}')
        
class Left(Top):
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Left for {cls}')
        
class Right(Top):
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Right for {cls}')
        
class Bottom(Left, Right):
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Bottom for {cls}')

Top for <class '__main__.Left'>
Top for <class '__main__.Right'>
Top for <class '__main__.Bottom'>
Right for <class '__main__.Bottom'>
Left for <class '__main__.Bottom'>
